# RAG Retrieval Enhanced with Document Summaries
In this section, we'll implement an improved approach to our retrieval system by incorporating document summaries. Instead of embedding chunks directly from the documents, we'll create a concise summary for each chunk and use this summary along with the original content in our embedding process.

This approach aims to capture the essence of each document chunk more effectively, potentially leading to improved retrieval performance.

Key steps in this process:

1. We load the original document chunks.
2. For each chunk, we generate a 2-3 sentence summary using OpenAI (or an OpenAI compatible API).
3. We store both the original content and the summary for each chunk in a new json file: data/anthropic_summary_indexed_docs.json

This summary-enhanced approach is designed to provide more context during the embedding and retrieval phases, potentially improving the system's ability to understand and match the most relevant documents to user queries.

In [1]:
## silent setup (-q), may take a while
!pip install openai -q
!pip install --upgrade tiktoken -q
!pip install pandas -q
!pip install numpy -q
!pip install matplotlib -q
!pip install seaborn -q
!pip install -U scikit-learn -q
!pip install sentence-transformers -q
!pip install pyyaml -q

In [2]:
# model configuration
embeddings_model = "intfloat/multilingual-e5-large-instruct"; generation_model = "gpt-4o-mini"; judge_model = "gpt-4o-mini"

In [3]:
import os
import getpass
from openai import OpenAI
OPENAI_API_KEY = getpass.getpass("Enter OpenAI API key")
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
# print(os.environ.get("OPENAI_API_KEY"))
client = OpenAI()

Enter OpenAI API key ········


In [4]:
from sentence_transformers import SentenceTransformer
embeddings_model = SentenceTransformer(embeddings_model)

/opt/vast-jupyter/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/128 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/140k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

### Generating the Summaries and Storing Them

In [5]:
# TODO, this is for Claud-3-haiku, need to be changed to OpenAI or Llama
import json
from tqdm import tqdm

def generate_summaries(input_file, output_file):
 
    # Load the original documents
    with open(input_file, 'r') as f:
        docs = json.load(f)

    # Prepare the context about the overall knowledge base
    knowledge_base_context = "This is documentation for Anthropic's, a frontier AI lab building Claude, an LLM that excels at a variety of general purpose tasks. These docs contain model details and documentation on Anthropic's APIs."

    summarized_docs = []

    for doc in tqdm(docs, desc="Generating summaries"):
        prompt = f"""
        You are tasked with creating a short summary of the following content from Anthropic's documentation. 

        Context about the knowledge base:
        {knowledge_base_context}

        Content to summarize:
        Heading: {doc['chunk_heading']}
        {doc['text']}

        Please provide a brief summary of the above content in 2-3 sentences. The summary should capture the key points and be concise. We will be using it as a key part of our search pipeline when answering user queries about this content. 

        Avoid using any preamble whatsoever in your response. Statements such as 'here is the summary' or 'the summary is as follows' are prohibited. You should get straight into the summary itself and be concise. Every word matters.
        """

        response = client.messages.create(
            model="claude-3-haiku-20240307",
            max_tokens=150,
            messages=[
                {"role": "user", "content": prompt}
            ],
            temperature=0
        )

        summary = response.content[0].text.strip()

        summarized_doc = {
            "chunk_link": doc["chunk_link"],
            "chunk_heading": doc["chunk_heading"],
            "text": doc["text"],
            "summary": summary
        }
        summarized_docs.append(summarized_doc)

    # Save the summarized documents to a new JSON file
    with open(output_file, 'w') as f:
        json.dump(summarized_docs, f, indent=2)

    print(f"Summaries generated and saved to {output_file}")
    
# this is already available, so the call is commented out
# generate_summaries('data/anthropic_docs.json', 'data/anthropic_summary_indexed_docs.json')

### Summary-Enhanced Vector Database Creation (heading + summary + chunk)
Here, we're creating a new vector database that incorporates our summary-enhanced document chunks. This approach combines the original text, the chunk heading, and the newly generated summary into a single text for embedding.

Key features of this process:

1. We create embeddings for the combined text (heading + summary + original content) using the Voyage AI API.
2. The embeddings and full metadata (including summaries) are stored in our vector database.
3. We implement caching mechanisms to improve efficiency in repeated queries.
4. The database is saved to disk for persistence and quick loading in future sessions.

This summary-enhanced approach aims to create more informative embeddings, potentially leading to more accurate and contextually relevant document retrieval.

In [6]:
import os
import numpy as np
import pickle
import json

class SummaryEnhancedVectorDB:
    def __init__(self, name, api_key=None):
        self.name = name
        self.embeddings = []
        self.metadata = []
        self.query_cache = {}
        self.db_path = f"./data/{name}/summary_indexed_vector_db.pkl"

    def _embed_and_store(self, texts, data):
        """not called for now"""
        batch_size = 128
        result = [
            embeddings_model.encode(texts[i : i + batch_size])
            for i in range(0, len(texts), batch_size)
        ]
        self.embeddings = [embedding for batch in result for embedding in batch]
        self.metadata = data
        
    def load_data(self, data_file):
        # Check if the vector database is already loaded
        if self.embeddings and self.metadata:
            print("Vector database is already loaded. Skipping data loading.")
            return
        # Check if vector_db.pkl exists
        if os.path.exists(self.db_path):
            print(f"Loading vector database from file: {self.db_path}.")
            self.load_db()
            return
            
        # well, if not...
        print(f'file {self.db_path} does not exist')
        with open(data_file, 'r') as f:
            data = json.load(f)

        texts = [f"{item['chunk_heading']}\n\n{item['text']}\n\n{item['summary']}" for item in data]  # Embed Chunk Heading + Text + Summary Together
        # Embed more than 128 documents with a for loop
        batch_size = 128
        result = [
            embeddings_model.encode(texts[i : i + batch_size])
            for i in range(0, len(texts), batch_size)
        ]

        # Flatten the embeddings
        self.embeddings = [embedding for batch in result for embedding in batch]
        self.metadata = data  # Store the entire item as metadata
        self.save_db()
        # Save the vector database to disk
        print("Vector database loaded and saved.")

    def search(self, query, k=3, similarity_threshold=0.75):
        query_embedding = None
        if query in self.query_cache:
            # print(f'found in cache!')
            query_embedding = np.array(self.query_cache[query])  #
            # print(f'type:{type(query_embedding)}')
        else:
            query_embedding = embeddings_model.encode(query)
            # print(f'query embedding:\n {query_embedding}')
            self.query_cache[query] = query_embedding.tolist()

        if not self.embeddings:
            raise ValueError("No data loaded in the vector database.")

        similarities = np.dot(self.embeddings, query_embedding)
        top_indices = np.argsort(similarities)[::-1]
        top_examples = []
        
        for idx in top_indices:
            if similarities[idx] >= similarity_threshold:
                example = {
                    "metadata": self.metadata[idx],
                    "similarity": similarities[idx],
                }
                top_examples.append(example)
                
                if len(top_examples) >= k:
                    break
        # self.save_db()
        return top_examples
    
    def save_db(self):
        data = {
            "embeddings": self.embeddings,
            "metadata": self.metadata,
            "query_cache": json.dumps(self.query_cache),
        }

        # Ensure the directory exists
        print(f'Saving DB in: {self.db_path}')
        os.makedirs(os.path.dirname(self.db_path), exist_ok=True)
        
        with open(self.db_path, "wb") as file:
            pickle.dump(data, file)

    def load_db(self):
        if not os.path.exists(self.db_path):
            raise ValueError("Vector database file not found. Use load_data to create a new database.")
        
        with open(self.db_path, "rb") as file:
            data = pickle.load(file)
        
        self.embeddings = data["embeddings"]
        self.metadata = data["metadata"]
        self.query_cache = json.loads(data["query_cache"])

In [7]:
#previewing our eval dataset
import json

def preview_json(file_path, num_items=3):
    try:
        with open(file_path, 'r') as file:
            data = json.load(file)
            
        if isinstance(data, list):
            preview_data = data[:num_items]
        elif isinstance(data, dict):
            preview_data = dict(list(data.items())[:num_items])
        else:
            print(f"Unexpected data type: {type(data)}. Cannot preview.")
            return
        
        print(f"Preview of the first {num_items} items from {file_path}:")
        print(json.dumps(preview_data, indent=2))
        print(f"\nTotal number of items: {len(data)}")
        
    except FileNotFoundError:
        print(f"File not found: {file_path}")
    except json.JSONDecodeError:
        print(f"Invalid JSON in file: {file_path}")
    except Exception as e:
        print(f"An error occurred: {str(e)}")

preview_json('evaluation/docs_evaluation_dataset.json')


Preview of the first 3 items from evaluation/docs_evaluation_dataset.json:
[
  {
    "id": "efc09699",
    "question": "How can you create multiple test cases for an evaluation in the Anthropic Evaluation tool?",
    "correct_chunks": [
      "https://docs.anthropic.com/en/docs/test-and-evaluate/eval-tool#creating-test-cases",
      "https://docs.anthropic.com/en/docs/build-with-claude/develop-tests#building-evals-and-test-cases"
    ],
    "correct_answer": "To create multiple test cases in the Anthropic Evaluation tool, click the 'Add Test Case' button, fill in values for each variable in your prompt, and repeat the process to create additional test case scenarios."
  },
  {
    "id": "1305ea00",
    "question": "What embeddings provider does Anthropic recommend for customized domain-specific models, and what capabilities does this provider offer?",
    "correct_chunks": [
      "https://docs.anthropic.com/en/docs/build-with-claude/embeddings#before-implementing-embeddings",
      "h

### Enhanced Retrieval Using Summary-Enhanced Embeddings
In this section, we implement the retrieval process using our new summary-enhanced vector database. This approach leverages the enhanced embeddings we created, which incorporate document summaries along with the original content.

Key aspects of this updated retrieval process:

1. We search the vector database using the query embedding, retrieving the top k most similar documents.
2. For each retrieved document, we include the chunk heading, summary, and full text in the context provided to the LLM.
3. This enriched context is then used to generate an answer to the user's query.

By including summaries in both the embedding and retrieval phases, we aim to provide the LLM with a more comprehensive and focused context. This could potentially lead to more accurate and relevant answers, as the LLM has access to both a concise overview (the summary) and the detailed information (the full text) for each relevant document chunk.

In [8]:
import json
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
from tqdm import tqdm
import logging
from typing import Callable, List, Dict, Any, Tuple, Set

def retrieve_similar_level_two(query, db):
    results = db.search(query, k=3)
    context = ""
    for result in results:
        chunk = result['metadata']
        context += f"\n <document> \n {chunk['chunk_heading']}\n\nText\n {chunk['text']} \n\nSummary: \n {chunk['summary']} \n </document> \n" #show model all 3 items
    return results, context

def construct_prompt(query, context):    
    prompt = f"""
    You have been tasked with helping us to answer the following query: 
    <query>
    {query}
    </query>
    You have access to the following documents which are meant to provide context as you answer the query:
    <documents>
    {context}
    </documents>
    Please remain faithful to the underlying context, and only deviate from it if you are 100% sure that you know the answer already. 
    Answer the question now, and avoid providing preamble such as 'Here is the answer', etc
    """

    return prompt

def answer_query_from_context_level_two(query, db):
    documents, context = retrieve_similar_level_two(query, db)
    completion = client.chat.completions.create(
    model=generation_model,
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": construct_prompt(query, context)
            }
        ],
        temperature=0.2
    )
    return completion.choices[0].message.content

# Load the evaluation dataset
with open('evaluation/docs_evaluation_dataset.json', 'r') as f:
    eval_data = json.load(f)

# Initialize the SummaryEnhancedVectorDB
level_two_db = SummaryEnhancedVectorDB("anthropic_docs_v2")
level_two_db.load_data('data/anthropic_summary_indexed_docs.json')
level_two_db.save_db()

# # Load the Anthropic documentation
# with open('data/anthropic_docs.json', 'r') as f:
#     anthropic_docs = json.load(f)

# test
query = "What embeddings provider does Anthropic recommend for customized domain-specific models, and what capabilities does this provider offer?"
test_results, test_contexts = retrieve_similar_level_two(query, level_two_db)
for i, test_result in enumerate(test_results):
    print(f'ith:{i}\n {test_result}')

file ./data/anthropic_docs_v2/summary_indexed_vector_db.pkl does not exist
Saving DB in: ./data/anthropic_docs_v2/summary_indexed_vector_db.pkl
Vector database loaded and saved.
Saving DB in: ./data/anthropic_docs_v2/summary_indexed_vector_db.pkl
ith:0
 {'metadata': {'chunk_link': 'https://docs.anthropic.com/en/docs/build-with-claude/embeddings#how-to-get-embeddings-with-anthropic', 'chunk_heading': 'How to get embeddings with Anthropic', 'text': 'How to get embeddings with Anthropic\n\n\nAnthropic does not offer its own embedding model. One embeddings provider that has a wide variety of options and capabilities encompassing all of the above considerations is Voyage AI.\nVoyage AI makes state-of-the-art embedding models and offers customized models for specific industry domains such as finance and healthcare, or bespoke fine-tuned models for individual customers.\nThe rest of this guide is for Voyage AI, but we encourage you to assess a variety of embeddings vendors to find the best fi

### Defining Our Metric Calculation Functions

In [12]:
def calculate_mrr(retrieved_links: List[str], correct_links: Set[str]) -> float:
    for i, link in enumerate(retrieved_links, 1):
        if link in correct_links:
            return 1 / i
    return 0

def evaluate_retrieval(retrieval_function: Callable, evaluation_data: List[Dict[str, Any]], db: Any) -> Tuple[float, float, float, float, List[float], List[float], List[float]]:
    precisions = []
    recalls = []
    mrrs = []
    
    for i, item in enumerate(tqdm(evaluation_data, desc="Evaluating Retrieval")):
        try:
            retrieved_chunks, _ = retrieval_function(item['question'], db)
            retrieved_links = [chunk['metadata'].get('chunk_link', chunk['metadata'].get('url', '')) for chunk in retrieved_chunks]
        except Exception as e:
            logging.error(f"Error in retrieval function: {e}")
            continue

        correct_links = set(item['correct_chunks'])
        
        true_positives = len(set(retrieved_links) & correct_links)
        precision = true_positives / len(retrieved_links) if retrieved_links else 0
        recall = true_positives / len(correct_links) if correct_links else 0
        mrr = calculate_mrr(retrieved_links, correct_links)
        
        precisions.append(precision)
        recalls.append(recall)
        mrrs.append(mrr)
        
        if (i + 1) % 10 == 0:
            print(f"Processed {i + 1}/{len(evaluation_data)} items. Current Avg Precision: {sum(precisions) / len(precisions):.4f}, Avg Recall: {sum(recalls) / len(recalls):.4f}, Avg MRR: {sum(mrrs) / len(mrrs):.4f}")
    
    avg_precision = sum(precisions) / len(precisions) if precisions else 0
    avg_recall = sum(recalls) / len(recalls) if recalls else 0
    avg_mrr = sum(mrrs) / len(mrrs) if mrrs else 0
    f1 = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall) if (avg_precision + avg_recall) > 0 else 0
    
    return avg_precision, avg_recall, avg_mrr, f1, precisions, recalls, mrrs

import tiktoken
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """For OpenAI models, returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

def evaluate_end_to_end(answer_query_function, db, eval_data):
    correct_answers = 0
    results = []
    total_questions = len(eval_data)
    
    for i, item in enumerate(tqdm(eval_data, desc="Evaluating End-to-End")):
        query = item['question']
        correct_answer = item['correct_answer']
        generated_answer = answer_query_function(query, db) # ??
        
        comparision_prompt = f"""
        You are an AI assistant tasked with evaluating the correctness of answers to questions about Anthropic's documentation.
        
        Question: {query}
        
        Correct Answer: {correct_answer}
        
        Generated Answer: {generated_answer}
        
        Is the Generated Answer correct based on the Correct Answer? You should pay attention to the substance of the answer, and ignore minute details that may differ. 
        
        Small differences or changes in wording don't matter. If the generated answer and correct answer are saying essentially the same thing then that generated answer should be marked correct. 
        
        However, if there is any critical piece of information which is missing from the generated answer in comparison to the correct answer, then we should mark this as incorrect. 
        
        Finally, if there are any direct contradictions between the correct answer and generated answer, we should deem the generated answer to be incorrect.
        
        Respond in the following XML format (don't prefix with xml):
        <evaluation>
        <content>
        <explanation>Your explanation here</explanation>
        <is_correct>true/false</is_correct>
        </content>
        </evaluation>
        """
        
        nb_tokens = num_tokens_from_string(comparision_prompt, "o200k_base")  # note, this encoding name for gpt-4o, gpt-4o-mini
        # print(f'Number of tokens: {nb_tokens}')
        
        try:
            response = client.chat.completions.create(
                model=judge_model,
                messages=[
                    {"role": "system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": comparision_prompt}
                ],
                temperature=0.2,
            )
            response_text = str(response.choices[0].message.content)
            print(f'Number of query tokens: {nb_tokens}, Query:\n{query}')
            print(f'__Correct answer__:\n{correct_answer}')
            print(f'__Generated answer__:\n{generated_answer}')
            print(f'__Response text from judge LLM__:\n{response_text}')
            
            evaluation = ET.fromstring(response_text)
            is_correct_value = evaluation.find(".//is_correct").text
            
            is_correct = is_correct_value == 'true'
            
            if is_correct:
                correct_answers += 1
            results.append(is_correct)
            
            logging.info(f"Question {i + 1}/{total_questions}: {query}")
            logging.info(f"Correct: {is_correct}")
            logging.info("---")
            
        except ET.ParseError as e:
            logging.error(f"XML parsing error: {e}")
            is_correct = 'true' in response_text.lower()
            results.append(is_correct)
        except Exception as e:
            logging.error(f"Unexpected error: {e}")
            results.append(False)
        
        if (i + 1) % 10 == 0:
            current_accuracy = correct_answers / (i + 1)
            print(f"Processed {i + 1}/{total_questions} questions. Current Accuracy: {current_accuracy:.4f}")
        # time.sleep(2)
    accuracy = correct_answers / total_questions
    return accuracy, results



In [13]:
# Initialize the SummaryIndexedVectorDB
# level_two_db = SummaryEnhancedVectorDB("anthropic_docs_v2")
# level_two_db.load_data('data/anthropic_summary_indexed_docs.json')

import pandas as pd

# Run the evaluations
eval_data_range = eval_data[:10]
avg_precision, avg_recall, avg_mrr, f1, precisions, recalls, mrrs  = evaluate_retrieval(retrieve_similar_level_two, eval_data_range, level_two_db)
e2e_accuracy, e2e_results = evaluate_end_to_end(answer_query_from_context_level_two, level_two_db, eval_data_range)

# Create a DataFrame
df = pd.DataFrame({
    'question': [item['question'] for item in eval_data_range],
    'retrieval_precision': precisions,
    'retrieval_recall': recalls,
    'retrieval_mrr': mrrs,
    'e2e_correct': e2e_results
})

# Save to CSV
from pathlib import Path
csv_dir = Path('evaluation/csvs')
csv_file_name = Path('evaluation_results_summary_enhanced.csv')
df.to_csv(csv_dir / csv_file_name, index=False)
print(f"Detailed results saved to {csv_dir/ csv_file_name}")

# Print the results
print(f"Average Precision: {avg_precision:.4f}")
print(f"Average Recall: {avg_recall:.4f}")
print(f"Average MRR: {avg_mrr:.4f}")
print(f"Average F1: {f1:.4f}")
print(f"End-to-End Accuracy: {e2e_accuracy:.4f}")

# Save the results to a json file
json_dir = Path("evaluation/json_results")
result_file_name = Path("evaluation_results_summary_enhanced.json")
Path(json_dir).mkdir(parents=True, exist_ok=True)
with open(json_dir / result_file_name, 'w') as f:
    json.dump({
        "name": "Summary Enhanced",
        "average_precision": avg_precision,
        "average_recall": avg_recall,
        "average_f1": f1,
        "average_mrr": avg_mrr,
        "end_to_end_accuracy": e2e_accuracy
    }, f, indent=2)

print(f"Evaluation complete. Results saved to {json_dir / result_file_name}, {csv_dir/ csv_file_name}")

Loading vector database from file: ./data/anthropic_docs_v2/summary_indexed_vector_db.pkl.


Evaluating Retrieval: 100%|██████████| 10/10 [00:00<00:00, 36.95it/s]


Processed 10/10 items. Current Avg Precision: 0.4667, Avg Recall: 0.7500, Avg MRR: 0.8000


Evaluating End-to-End:  10%|█         | 1/10 [00:04<00:37,  4.17s/it]

Number of query tokens: 506, Query:
How can you create multiple test cases for an evaluation in the Anthropic Evaluation tool?
__Correct answer__:
To create multiple test cases in the Anthropic Evaluation tool, click the 'Add Test Case' button, fill in values for each variable in your prompt, and repeat the process to create additional test case scenarios.
__Generated answer__:
To create multiple test cases for evaluation in the Anthropic Evaluation tool, follow these steps:

1. **Access the Evaluation Tool**: Open the Anthropic Console and navigate to the prompt editor. After composing your prompt, look for the ‘Evaluate’ tab at the top of the screen.

2. **Prepare Your Prompt**: Ensure your prompt includes at least 1-2 dynamic variables using the double brace syntax (e.g., {{variable}}). This is necessary for creating evaluation test sets.

3. **Add Test Cases**: On the Evaluation screen, you will see a single row for your initial test case. To add more test cases, click the ‘Add Tes

...

Number of query tokens: 428, Query:
When deciding whether to use chain-of-thought (CoT) for a task, what are two key factors to consider in order to strike the right balance between performance and latency?
__Correct answer__:
When deciding whether to use CoT, consider if the task requires in-depth thinking that a human would need to work through, and be aware that the increased output length from CoT may impact latency.
__Generated answer__:
When deciding whether to use chain-of-thought (CoT) for a task, two key factors to consider are:

1. **Task Complexity**: CoT should be used for tasks that require in-depth thinking, such as complex math, multi-step analysis, or decisions with many factors. If the task does not necessitate extensive processing, it may be better to avoid CoT to maintain efficiency.

2. **Latency Impact**: The use of CoT can increase output length, which may negatively impact latency. It's important to weigh the benefits of improved performance against the potential

Evaluating End-to-End:  90%|█████████ | 9/10 [00:46<00:05,  5.62s/it]

Number of query tokens: 631, Query:
How can I use Claude to more easily digest the content of long PDF documents?
__Correct answer__:
You can upload PDFs and have Claude summarize their content, making it easier to understand the key points of long documents without having to read through everything.
__Generated answer__:
To use Claude for digesting long PDF documents more effectively, follow these strategies:

1. **Place Longform Data at the Top**: Start your prompt with the content of the PDF document. This ensures that Claude processes the most relevant information first, which can significantly enhance its performance, especially with lengthy documents (around 20K+ tokens).

2. **Structure Content with XML Tags**: If you're working with multiple documents, organize the content using XML tags. For example, wrap each document in `<document>` tags and include `<document_content>` and `<source>` tags for clarity. This structured approach helps Claude understand the context better.

   

Evaluating End-to-End: 100%|██████████| 10/10 [00:48<00:00,  4.86s/it]

Number of query tokens: 276, Query:
According to the documentation, where can you view your organization's current API rate limits in the Anthropic Console?
__Correct answer__:
You can view your organization's current API rate limits in the Rate Limits tab of the Developer Console.
__Generated answer__:
You can view your organization's current API rate limits in the new Rate Limits tab in the Developer Console.
__Response text from judge LLM__:
<evaluation>
<content>
<explanation>The generated answer states that you can view your organization's current API rate limits in the "new Rate Limits tab" in the Developer Console, which is essentially the same information as the correct answer that mentions the "Rate Limits tab of the Developer Console." The addition of "new" does not change the substance of the answer, so it is still correct.</explanation>
<is_correct>true</is_correct>
</content>
</evaluation>
Processed 10/10 questions. Current Accuracy: 0.8000
Detailed results saved to evalua

In [11]:
!cat evaluation/json_results/evaluation_results_summary_enhanced.json 

{
  "name": "Summary Enhanced",
  "average_precision": 0.4666666666666666,
  "average_recall": 0.75,
  "average_f1": 0.5753424657534246,
  "average_mrr": 0.8,
  "end_to_end_accuracy": 0.7
}

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
